## Lectura de archivos de alineamientos múltiples de secuencias.

El módulo Bio.AlignIO contiene funciones que permiten leer archivos de MSA.

La función *read* lee el contenido de un archivo.
Se lo puede convertir en una lista para acceder a cada secuencia.
Es necesario especificar el formato de los archivos de entrada.

In [ ]:
from Bio import AlignIO

seqs = AlignIO.read("data/some_sequences.fasta", format="fasta")
seqs = list(seqs)
print(seqs)

Cada secuencia individual se representa como un objeto de tipo *SeqRecord*.

* Un *SeqRecord* tiene dos atributos importantes: *id* y *seq*. 
 * El *id* es un string que contiene el nombre o identificador de la secuencia, 
 * y *seq* es un objeto de tipo *Seq* que contiene los datos de la secuencia.
* Un objeto *Seq* tiene la secuencia propiamente dicha y un alfabeto.


In [ ]:
s1 = seqs[0]
print("s1 es la primer secuencia")
print("s1 es de tipo: " + str(type(s1)))
print("El id de s1 es: " + s1.id)
print("La secuencia de s1 es de tipo: " + str(type(s1.seq)))
print("La secuencia de s1 es: " + s1.seq)
print("La secuencia de s1 tiene un el alfabeto: " + str(s1.seq.alphabet))

**Desafío**: Imprima la secuencia de la segunda file del MSA

### Crear un objeto *Seq*

In [ ]:
from Bio.Seq import Seq
from Bio.Seq import translate
from Bio.Alphabet import DNAAlphabet
from Bio.Alphabet import ProteinAlphabet

new_seq = Seq("ACTACTGACTAC", DNAAlphabet())
print(new_seq)

**Desafío**: Cree un objeto *Seq* para una secuencia de una proteína.

### Exportar secuencias a un archivo

In [ ]:
from Bio import AlignIO
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

s1 = SeqRecord(id="Secuencia1", seq=Seq("ACTATCTAGCTACTG"))
s2 = SeqRecord(id="Secuencia2", seq=Seq("CTATCTAGCTACTGA"))
s3 = SeqRecord(id="Secuencia3", seq=Seq("TATCTAGCTACTGAA"))
s4 = SeqRecord(id="Secuencia4", seq=Seq("ATCTAGCTACTGAAA"))

secuencias = [s1, s2, s3, s4]

SeqIO.write(secuencias, 'secuencias_de_ejemplo.fasta', format="fasta")
SeqIO.write(secuencias, 'secuencias_de_ejemplo.clustal', format="clustal")

# acepta muchos otros formato:
# phylip, nexus, stockholm, etc

**Desafío**: Cree dos secuencias nuevas y agreguelas a la lista *secuencias*. Exporte el MSA en formato 'phylip'.

### Convertir el formato de un archivo

La función *convert* permite rápidamente convertir el formato entre archivos.

In [ ]:
SeqIO.convert('secuencias_de_ejemplo.fasta', 'fasta', 'secuencias_de_ejemplo.stockholm', 'stockholm')

**Desafío**: Convierta el MSA a formato 'clustal'.

### Operaciones sobre secuencias

In [ ]:
from Bio.Seq import Seq
from Bio.Seq import translate
from Bio.Alphabet import DNAAlphabet

new_seq = Seq("ATGTTAGCTTGA", DNAAlphabet())

translated = new_seq.translate()
print("translated : " + translated)

transcribed = new_seq.transcribe()
print("transcribed: "+transcribed)

complement = new_seq.complement()
print("complement : "+complement)

rev_complement = new_seq.reverse_complement()
print("complement : "+rev_complement)

new_seq = Seq("---ATGTTAGCTTGA---", DNAAlphabet())
degapped = new_seq.ungap("-")
print("ungapped : " + degapped)

**Desafío**: Cree una nueva secuencia de DNA, traduzcala a proteína y luego transcribala a RNA. 

## Alineamiento de a pares de secuencias

Se quiere alinear estas dos secuencias.

<pre>
Seq1 = ATCATCTACTGCTGACTCTTGA
Seq2 = ATCATCTACTCACTCTG
</pre>

### Características de un alineamiento

<pre>
Seq1  ATCATCT A CTG CTGACTCT T GA
            | | ||     ||||| 
Seq2  ATCATCT A CTC ---ACTCT G --
              ^     ^^       ^
match ---------     ||       |
gap open ------------|       |
gap extension --------       |
mismatch ---------------------
</pre>

Es posible asignar un puntaje a cada *matches*, *mismatches*, *gap opening* y *gap extension*.

El objetivo es encontrar una correspondencia entre los caracteres de las dos secuencias tal que maximice la suma de los puntajes anteriores.

### Tipos de alineamiento

* Alineamientos globales.
Un alineamiento global encuentra la mejor forma de aparear todos los caracteres de dos secuencias.

* Alineamientos locales.
Un alineamiento local busca la mejor subsecuencia que alinea entre dos secuencias.

#### Puntajes para *match* y *mismatch*

| Código | Descripción |
| --- | --- |
| x | Carácteres idénticos tienen un puntaje de 1, sino es 0 |
| m | Se define un puntaje para un *match* y otro para un *mismatch* |
| d | Se define un puntaje diferente para todos los pares de caracteres |
| c | Se define una función que calcular el puntaje |

#### Puntajes para *gaps*

| Código | Descripción |
| --- | --- |
| x | No se penalizan los gaps. |
| s | Se define un puntaje de penalización para abrir y otro para extender *gaps*. Es el mismo para las dos secuencias|
| d | Cada secuencia tiene puntajes diferentes de penzalización para abrir y para extender *gaps*.  |
| c | Se define una función que calcular el puntaje. |


### Alineamiento simple

In [ ]:
from Bio.pairwise2 import align
from Bio.pairwise2 import format_alignment

seq1 = "CTCAHYRKLAMNFQAW"
seq2 = "YRKIAMNF"

aln = align.globalxx(seq1, seq2)

print("aln contiene " + str(len(aln)) + " alineamientos.")

align1, align2, score, begin, end = aln[0]
print(format_alignment(align1, align2, score, begin, end))

align1, align2, score, begin, end = aln[1]
print(format_alignment(align1, align2, score, begin, end))

**Desafío**: Cree una secuencia *seq3* que al alinearse contra *seq1* devuelv una alineamiento con tres gaps internos. Imprime el resultado del alineamiento.

### Alineamiento con penalización de *gaps*

In [ ]:
seq1 = "CTCAHYRKLAMNFQAW"
seq2 = "YRKIAMNF"
gap_opening = -0.5
gap_extension = -0.1
aln = align.globalxs(seq1, seq2, gap_opening, gap_extension)
print("aln contiene " + str(len(aln)) + " alineamientos.")
print(format_alignment(*aln[0]))

**Desafío**: Rehaga el alineamiento cambiando los valores de penalización, hasta que encontrar un alineamiento que solo contenga un único bloque de *gaps*

### Diferencia entre local y global

In [ ]:
seq1 = "CTCAHYRKLAMNFQAWYCAYHCACYHACYHACHACHYACYAYCAYCYANNANCAKLKLMNMCNA"
seq2 = "CTMNYRKIAMNFAYCAYCYANNANC"

match = 2
mismatch = -1
gap_opening = -1
gap_extension = -0.5

aln_global = align.globalms(seq1, seq2, match, mismatch, gap_opening, gap_extension)
print(format_alignment(*aln_global[0]))

aln_local = align.localms(seq1, seq2,  match, mismatch, gap_opening, gap_extension)
print(format_alignment(*aln_local[0]))

**Desafío**: Cambio las penalizaciones de *gaps* hasta que los dos alineamientos coincidan.

### Alineamiento usando una matriz de sustitución

In [ ]:
from Bio.SubsMat import MatrixInfo as matlist

print("Matrices de score que pueden usarse:" + str(matlist.available_matrices)+"\n\n")

seq1 = "YRKLAENF"
seq2 = "FKRIVDQY"
matrix = matlist.blosum62

print("Alineamiento con puntajes de match/mismatch ")
for a in align.globalms(seq1, seq2, 2, -1, -0.7, -0.1):
    print(format_alignment(*a))

print("Alineamiento con la matriz Blosum62")
for a in align.globalds(seq1, seq2, matrix, -0.7, -0.1):
    print(format_alignment(*a))

**Desafío**: Rehaga el alineamiento usando las matrices 'pam30', 'blosum95' y otra matrices de su elección.

In [ ]:
print("Alineamiento con la matriz Blosum62")
for a in align.globalds(seq1, seq2, matlist.blosum95, -0.7, -0.1):
    print(format_alignment(*a))

## Leer archivos PDB

Un PDB tiene una jerarquía de elementos que definen cada componente de la estuctura 3D de una mocromolécula.

* Un PDB -> Contiene una Estructura
* Estructura -> Contiene uno o más modelos. 
* Modelo -> Contiene una o más cadenas.
* Cadena -> Contiene uno o más residuos.
* Residuo -> Contiene uno o más átomos.
* Átomo -> Elemeton base de la jerarquía.

In [ ]:
from Bio.PDB import PDBParser

parser = PDBParser()
structure = parser.get_structure('6H9U', 'data/6h9u.pdb')

for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                print("model: " + str(model.id),
                      "Chain: " + str(chain.id), 
                      "Residue:" + str(residue.resname) + "-" + str(residue.id),
                      "Atom: "+ str(atom.id) +" "+ str(atom.coord))
                



**Desafío**: Cuente cuantos átomos tiene cada cadena.

### Calcular distancia entre dos átomos

In [ ]:
atom1 = structure[0]['A'][100]['CA']
atom2 = structure[0]['A'][99]['CA']
print(atom1, atom2, atom1-atom2)

### Obtener la secuencia de una cadena de PDB

In [ ]:
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import PPBuilder

parser = PDBParser()
structure = parser.get_structure('6H9U', 'data/6h9u.pdb')

chain = structure[0]['A']
polypeotides = PPBuilder().build_peptides(chain)
for polypeptide in polypeotides:
    print(polypeptide.get_sequence()+"\n")


**Desafío**: Obtenga la secuencia de la cadena 'B'.

### Visualizar un PDB

In [ ]:
import nglview
view = nglview.show_file('data/6h9u.pdb')
view